In [17]:
import pandas as pd
import numpy as np
import joblib
import argparse
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model and confusion matrix
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn import tree
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random

In [67]:
# LOAD THE MODEL
common_dir = "/home/giulia/Workspace/PhytoPhD/period_abroad/"
# eff_predictions = f"{common_dir}random_forest/tmp/old_validation_pre_nizza/feature_table_TNS.tsv"
eff_predictions = f"{common_dir}random_forest/on_dataset20230215/pred_validation/feature_table_eff25_58_val.tsv"
non_eff_predictions = f"{common_dir}random_forest/on_dataset20230215/pred_validation/feature_table_noneff_val.tsv"
loaded_model = joblib.load(f"{common_dir}random_forest/on_dataset20230214/rf_training/rf_best_model.pkl")

In [62]:
features = ["name", "sequence length", "signal peptide", "transmembrane domain", 
                                 "aa in tr domain", "first 60 aa", "prob N-in", "warning signal sequence", 
                                 "MobiDB-lite", "CLUMP0", "CLUMP1", "CLUMP5", 
                                 "CLUMP6", "CLUMP7", "CLUMP8","bin1", "bin2", "bin3", "bin4"]

In [100]:
# FOR SINGLE CLASS USAGE 
class_name = "effectors"
class_file = "/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/prediction_tools/pred_dataset_validazione20221110/feature_table_eff/feature_table_eff_truly_never_seen20221115.tsv"

data_pred_df_original = pd.read_csv(class_file, sep="\t", header=0)
data_pred_df = pd.read_csv(class_file, sep="\t", header=0)
data_pred_df["name"] = [class_name] * len(data_pred_df)
data_pred_df.replace([True, False], ["1", "0"], inplace=True)

In [ ]:
# PUTATIVE EFFECTORS EXPLICITED
eff_pred_df_original = pd.read_csv(eff_predictions, sep="\t", header=0)
non_eff_pred_df_original = pd.read_csv(non_eff_predictions, sep="\t", header=0)
data_pred_df_original = pd.concat([eff_pred_df_original, non_eff_pred_df_original], ignore_index=True, join="inner")
data_pred_df_original.replace([True, False], ["1", "0"], inplace=True)

print(data_pred_df_original)
ids = []

with open("/home/giulia/Workspace/PhytoPhD/effectors_analysis/dataset_validazione20221110/uniprot-download_true_format_list_query__28Putative_29-2022.11.15-15.57.14.63.list", 
          "r") as only_putative_ids: 
    asd = only_putative_ids.readlines()
    for el in asd:
        ids.append(el.replace("\n", ""))
    print(ids)

In [55]:
# CONCAT EFF non-EFF VALIDATION SET 

original_protein_names = pd.concat([pd.read_csv(eff_predictions, sep="\t", header=0)[["name"]], 
                                    pd.read_csv(non_eff_predictions, sep="\t", header=0)[["name"]]], 
                                   ignore_index=True, join="inner")

## EFF PREDICTIONS DF for Random Forest
eff_pred_df = pd.read_csv(eff_predictions, sep="\t", header=0)
# eff_pred_df["name"] = ["effector"] * len(eff_pred_df)

## NON-EFF PREDICTIONS DF for Random Forest
non_eff_pred_df = pd.read_csv(non_eff_predictions, sep="\t", header=0)
# non_eff_pred_df["name"] = ["non_effector"] * len(non_eff_pred_df)

## COMBINE DFs (taking only effector motifs)
for col in eff_pred_df.columns[1:]:
    if col not in non_eff_pred_df.columns[1:]:
        # if an effector motifs is not present in non effectors, a new column will be added to the non_effector_df
        # having value False or 0 in this case
        non_eff_pred_df[col] = [0] * len(non_eff_pred_df)
    else:
        pass

for feature in features:
    if feature not in list(eff_pred_df.columns):
        eff_pred_df[feature] = [0] * len(eff_pred_df)
    if feature not in list(non_eff_pred_df.columns):
        non_eff_pred_df[feature] = [0] * len(non_eff_pred_df)
        
# CONCAT AND CLEAN UP THE DATAFRAME FOR THE TRAINING
# "inner" will take only common cols
data_pred_df = pd.concat([eff_pred_df, non_eff_pred_df], ignore_index=True, join="inner")
data_pred_df.replace([True, False], ["1", "0"], inplace=True)
data_pred_df = data_pred_df[data_pred_df.columns[1:]]

print(len(features))
print(data_pred_df)

19
             name  sequence length  signal peptide  transmembrane domain  \
0        effector              112           0.886                     1   
1        effector              112           0.886                     1   
2        effector              112           0.886                     1   
3        effector              112           0.886                     1   
4        effector              112           0.886                     1   
..            ...              ...             ...                   ...   
523  non_effector              732           0.628                     1   
524  non_effector              357           0.000                     1   
525  non_effector              145           0.691                     1   
526  non_effector              135           0.000                     1   
527  non_effector              635           0.648                     1   

     aa in tr domain  first 60 aa  prob N-in  warning signal sequence  \
0          

In [101]:
# PARSE THE REAL DATA

# data_pred_df = pd.read_csv(data_predictions, sep="\t", header=0)
# data_pred_df.replace([True, False], ["1", "0"], inplace=True)

cols_to_drop = []

for col in list(data_pred_df.columns):
    if col not in features:
        cols_to_drop.append(col)
    else:
        pass
    
data_pred_df = data_pred_df.drop(columns=cols_to_drop, axis=1)

for col in features:
    if col not in data_pred_df.columns:
        data_pred_df[col] = [0] * len(data_pred_df)
    else:
        pass
    
data_pred_df = data_pred_df[features]
len(data_pred_df)

58

In [ ]:
# PROB ON PREDICITONS
print(loaded_model.feature_names_in_)
y_proba = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
pred_proba_df = pd.DataFrame(y_proba, columns=loaded_model.classes_)

pred_proba_df["name"] = original_protein_names
pred_proba_df["known_clf"] = data_pred_df["name"]
pred_proba_df.to_csv(f"{common_dir}random_forest/rf_clumps_dataset_validazione20230131/predictions_probability_on_effTNS_negrandom100_20221116_20230131.tsv",
                    sep="\t",
                    index=True)

pred_proba_df

In [9]:
# APPLY TO REAL DATA

# data_pred_df_putative_exp_col = []
# ids = []
# for i in range(len(data_pred_df_original)):
#     if data_pred_df_original["name"].iloc[i] in ids:
#         data_pred_df_putative_exp_col.append("putative_effectors")
#     else:
#         data_pred_df_putative_exp_col.append("effectors")
loaded_model = joblib.load(f"{base_dir}on_dataset20230224/clumps_features/rf_best_model.pkl")
clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
list_clf = list(clf)

# clf_df = pd.DataFrame(list(zip(list(original_protein_names), list(data_pred_df["name"]), clf)),
#                               columns=["uniprot_name", "class", "RF_model_classification"])

# print(clf_df["RF_model_classification"].value_counts())
# clf_df.to_csv(f"{common_dir}random_forest/rf_clumps_dataset_validazione20230213/clf_on_effTNS_negrandom100fake_20221116.tsv",
#              sep="\t",
#              index=False)
# print(data_pred_df["name"], clf)
acc = metrics.accuracy_score(data_pred_df["name"], clf)
conf_matrix = metrics.confusion_matrix(data_pred_df["name"], clf)
known_hot_encode = [0 if el == "non_effector" else 1 for el in data_pred_df["name"]]
clf_hot_encode = [0 if el == "non_effector" else 1 for el in clf]
precision_perc = metrics.precision_score(known_hot_encode, clf_hot_encode, pos_label=1) # calculated on positive class
recall_perc = metrics.recall_score(known_hot_encode, clf_hot_encode, pos_label=1) # calculated on positive class
print(f"accuracy: {acc}")
print(f"conf_matrix:\n{conf_matrix}")
print(f"precision: {precision_perc}")
print(f"recall: {recall_perc}")
pd.DataFrame({"clf": list_clf, "real": list(data_pred_df["name"]), "seq_id": list(origin_data_pred_df["seq_id"])}).to_csv(f"{base_dir}on_uniproteome_mali20230214/rf_validation/protein_classification.tsv")
# with open(f"{common_dir}random_forest/on_dataset20230214/rf_validation/summary_performances.txt", "w") as out:
# #     out.write(f"{list(clf_df['RF_model_classification'].value_counts().index)[0]}\t{clf_df['RF_model_classification'].value_counts()[0]}\n")
# #     out.write(f"{list(clf_df['RF_model_classification'].value_counts().index)[1]}\t{clf_df['RF_model_classification'].value_counts()[1]}\n")
#     out.write(f"accuracy\t{acc}\nconf_matrix:\n{conf_matrix}\nprecision\t{precision_perc}\nrecall\t{recall_perc}")    

     sequence length  signal peptide  transmembrane domain  aa in tr domain  \
0                741           0.000                     0          0.00000   
1                394           0.000                     0          0.22113   
2                211           0.000                     0          0.04486   
3                501           0.678                     1         20.64188   
4                263           0.700                     1         20.77388   
..               ...             ...                   ...              ...   
514              233           0.000                     5        109.90518   
515              286           0.735                     1         20.30865   
516              354           0.000                     0          0.00422   
517               54           0.000                     0          0.00018   
518              329           0.000                     0          0.00237   

     first 60 aa  prob N-in  warning signal sequenc

In [54]:
## unique feature table 
base_dir = "/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/"
origin_data_pred_df = pd.read_csv(f"{base_dir}on_7_strains/capm40/pred/feature_table_capm40_clumps_std.tsv",
                          sep="\t", header=0)
original_protein_names = origin_data_pred_df["name"]

data_pred_df = origin_data_pred_df[origin_data_pred_df.columns[1:]] # df without seq_id 
data_pred_df

,name,sequence length,signal peptide,transmembrane domain,aa in tr domain,first 60 aa,prob N-in,warning signal sequence,MobiDB-lite,CAMP_PHOSPHO_SITE,...,CLUMP0,CLUMP3,CLUMP5,CLUMP6,CLUMP7,CLUMP8,bin1,bin2,bin3,bin4
0,to_be_predicted,356,0.000,1,21.91817,21.77976,0.79473,1,0,0,...,1,0,1,0,3,0,5,0,0,1
1,to_be_predicted,842,0.628,1,20.23306,20.23306,0.99993,1,0,0,...,2,1,0,2,6,1,3,5,2,10
2,to_be_predicted,67,0.574,1,21.91432,21.90733,0.03253,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,to_be_predicted,132,0.736,0,10.61466,10.61226,0.41875,1,23,0,...,0,0,0,0,2,0,0,1,0,1
4,to_be_predicted,601,0.637,2,45.16399,22.44981,0.00499,1,0,0,...,1,0,1,1,3,1,2,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,to_be_predicted,75,0.000,0,0.00056,0.00056,0.21904,0,0,1,...,0,0,0,0,1,0,0,0,1,0
542,to_be_predicted,267,0.000,7,154.50996,22.62559,0.98469,1,0,0,...,0,0,0,0,2,0,2,0,1,0
543,to_be_predicted,232,0.807,1,34.07659,12.43419,0.60223,1,0,0,...,0,0,0,0,2,0,1,0,0,2
544,to_be_predicted,30,0.000,0,0.00000,0.00000,0.80363,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
## APPLY AFTER CROSS VALIDATION 
# basic names for application
model_feats = "clumps_std"
bn_app = "capm40"
bn_prot = "CaPm40"

# load and apply the model
loaded_model = joblib.load(f"{base_dir}on_dataset20230224/{model_feats}_features/rf_best_model.pkl")
clf = loaded_model.predict(data_pred_df[list(data_pred_df.columns)[1:]])
list_clf = list(clf)

# probability of class assignment 
prob = loaded_model.predict_proba(data_pred_df[list(data_pred_df.columns)[1:]])
df_prob = pd.DataFrame(prob, columns=["effector_prob", "non_effector_prob"])
df_prob["seq_id"] = list(origin_data_pred_df["seq_id"])
df_prob.to_csv(f"{base_dir}on_7_strains/{bn_app}/model_{model_feats}_on_dataset20230224/probability_of_predictions.tsv", sep="\t", index=False)
print(len(df_prob[df_prob["effector_prob"] > 0.9]))
print(len(df_prob[df_prob["effector_prob"] > 0.9])/len(origin_data_pred_df))
highly_eff = df_prob[df_prob["effector_prob"] > 0.9]
with open(f"{base_dir}on_7_strains/{bn_app}/model_{model_feats}_on_dataset20230224/effector_predictions_filt90.fasta", "w") as filtered_seq:
    for record in SeqIO.parse(f"/home/giulia/Workspace/PhytoPhD/period_abroad/datasets/7_strains/{bn_prot}_prodigal_faa.fasta", "fasta"):
        if record.id in list(highly_eff["seq_id"]):
            filtered_seq.write(f">{record.description}\n{str(record.seq)}\n")

# save the summary of performances 
with open(f"{base_dir}on_7_strains/{bn_app}/model_{model_feats}_on_dataset20230224/prediction_summary.txt", "w") as out:
    count_eff = list_clf.count('effector')
    count_noneff = list_clf.count('non_effector')
    out.write(f"Predicted {count_eff} effectors and {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}")

    
# save the predictions as they are
df_predictions = pd.DataFrame({"clf": list_clf, "protein_id": origin_data_pred_df["seq_id"]})
df_predictions.to_csv(f"{base_dir}on_7_strains/{bn_app}/model_{model_feats}_on_dataset20230224/predictions.tsv", sep="\t", index=False)
    
print(f"Predicted {count_eff} effectors and {count_noneff} non-effectors\nProportion of effectors/total proteins: {(count_eff/count_noneff)*100}%")

74
0.13553113553113552
Predicted 193 effectors and 353 non-effectors
Proportion of effectors/total proteins: 54.6742209631728%


In [8]:
########################################## OLD #################################################################
# APPLY TO known effectors DATA
known = ["effectors"] * len(pred_data_eff)
clf = loaded_model.predict(pred_data_eff[list(pred_data_eff.columns)[1:]])
clf_df = pd.DataFrame(list(zip(list(pred_data_eff["name"]), clf)),
                              columns=["name", "RF_model_classification"])

print(clf_df["RF_model_classification"].value_counts())
# clf_df.to_csv("/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/random_forest/rf_dataset_validazione20220805/putative_eff_eff_clf.tsv",
#              sep="\t",
#              index=False)


acc = metrics.accuracy_score(known, clf)
conf_matrix = metrics.confusion_matrix(known, clf)
known_hot_encode = [0 if el == "non_effectors" else 1 for el in known]
clf_hot_encode = [0 if el == "non_effectors" else 1 for el in clf]
precision_perc = metrics.precision_score(known_hot_encode, clf_hot_encode, pos_label=1)
recall_perc = metrics.recall_score(known_hot_encode, clf_hot_encode, pos_label=1)
print(f"accuracy: {acc}")
print(f"conf_matrix:\n{conf_matrix}")
print(f"precision: {precision_perc}")
print(f"recall: {recall_perc}")

effectors        26
non_effectors    12
Name: RF_model_classification, dtype: int64
accuracy: 0.6842105263157895
conf_matrix:
[[26 12]
 [ 0  0]]
precision: 1.0
recall: 0.6842105263157895


In [ ]:
clf_fit = loaded_model.fit(pred_data_eff[list(pred_data_eff.columns)[1:]], pred_data_eff["name"])
perm_import = permutation_importance(clf_fit, pred_data_eff[list(pred_data_eff.columns)[1:]], pred_data_eff["name"],
                                    n_repeats=30, random_state=42, scoring='accuracy')
perm_import

In [ ]:
### HYDROPHOBIC PROFILE
l = np.array([1, 2, 4, 10, 53, 8, 20, 88])
l1 = np.array([5, 7, 8, 1])

l_mean = l.mean()
l_std = l.std()
l1_mean = l1.mean()
l1_std = l1.std()

# l_1 = (l*l_mean)/l_std
# l1_1 = (l1*l1_mean)/l1_std

l = ((l-l.min()) / (l.max() - l.min())) / len(l)
l1 = ((l1-l1.min()) / (l1.max()-l.min())) / len(l1)

# l = l/len(l)
# l1 = l1/len(l1)
bins = [[l[:5].mean(), l[5:].mean()], [l1[:3].mean(), l1[3:].mean()]]
print(l)
print(l1)
# print(l_1)
# print(l1_1)
print(bins)

In [20]:
# effectors 
# no direct predictions, take the corresponding rows from the putative_eff_eff feature table
data_eff = SeqIO.parse("/home/giulia/Workspace/PhytoPhD/effectors_analysis/dataset_validazione20220805/uniprot_effectors_never_seen_byRF20220805.fasta", "fasta")
name_eff = []
for record in data_eff:
    name = record.id.split("|")[1]
    name_eff.append(name)

name_eff_putative_eff = list(data_pred_df["name"])
index = []
for el in name_eff:
    if el in name_eff_putative_eff:
        index.append(name_eff_putative_eff.index(el))
        
pred_data_eff = data_pred_df.iloc[index]

len(pred_data_eff)


          name  sequence length  signal peptide  aa in tr domain  first 60 aa  \
0   A0A0G7ZN61              302           0.405          0.08054      0.00038   
1   A0A1Q1NH89              112           0.886         22.14889     22.14889   
2   A0A1Q1NH90              112           0.886         22.14889     22.14889   
3   A0A1Q1NH91              112           0.886         22.14889     22.14889   
4   A0A1Q1NH92              112           0.886         22.14889     22.14889   
5   A0A1Q1NH98              112           0.886         22.14889     22.14889   
6   A0A1Q1NH99              112           0.886         22.14889     22.14889   
7   A0A1Q1NHA1              112           0.886         22.14889     22.14889   
8   A0A1Q1NHA2              112           0.886         22.14889     22.14889   
9   A0A1Q1NHA3              112           0.886         22.14889     22.14889   
10  A0A1Q1NHA5              112           0.886         22.14889     22.14889   
11  A0A1Q1NHA6              

38